# Quick check on language swapping  across Wikipedia editions

In this notebook we check users visiting more than one language edition in a given day. 
Our approach have the following limitations/contrains:

* User is approached by concatenating user_agent and IP. 
* We just consider users visiting Wikipedia (other projects like Wikidata are excluded)
* We use the webrequest table, that only contains information from the last 90 days (longitudinal studies would require to track this information along time)
* We work on a random sample of users (~10%).


## Number of projects visited by users between Dec 1th and Dec 5th, 2018

* Value cnt reflects the % of users visiting that amount of editions. Hence, cnt=1, percent= 94.1, means that the 94.1% of users had just visited one edition, implying that just 5.9% users visted more than one edition.

In [1]:
sqlContext.sql('use wmf')

DataFrame[]

In [11]:
for day in range(1,5):
    ## QUERY FOR A GIVEN DAY. 
    ## sampling around 10% of users
    query = '''
    SELECT CONCAT(user_agent,client_ip) as user,   normalized_host.project, COUNT(normalized_host.project) as projectCount
    FROM webrequest WHERE normalized_host.project_family = "wikipedia" AND is_pageview = 1 AND agent_type="user" 
    AND year=2018 AND month = 12 and day = %s AND SUBSTR(ip,-1,1) = 5 
    GROUP BY user, normalized_host
    ''' % str(day)
    df = sqlContext.sql(query)
    sqlContext.registerDataFrameAsTable(df, "users")
    dfUsers = sqlContext.sql('SELECT user,COUNT(project) as cnt FROM users GROUP BY user SORT BY cnt')
    sqlContext.registerDataFrameAsTable(dfUsers, "usersCounts")    
    totalUsers = dfUsers.count()
    print(totalUsers,'users in sample',day,'Dec 2018')
    final = sqlContext.sql('SELECT cnt,100*COUNT(cnt)/%s  as percent FROM usersCounts GROUP BY cnt SORT BY cnt DESC' % totalUsers)
    final.sort('cnt').show()
    results[country] = final.toPandas()

12847626 users in sample 1 Dec 2018
+---+--------------------+
|cnt|             percent|
+---+--------------------+
|  1|   94.83578522600206|
|  2|   4.656502298556948|
|  3|  0.3842266267713584|
|  4| 0.07050329765203314|
|  5| 0.02170050715984416|
|  6|0.010492210778862959|
|  7|0.005448477407421418|
|  8|0.003315787679373...|
|  9|0.002179390962968567|
| 10| 0.00309006504392329|
| 11|9.340246984151002E-4|
| 12|6.616008280440293E-4|
| 13|4.670123492075501E-4|
| 14|3.269086444452850...|
| 15|2.957744878314484E-4|
| 16|2.490732529106934E-4|
| 17|2.490732529106934E-4|
| 18|2.412897137572342...|
| 19|2.023720179899383...|
| 20|1.089695481484283...|
+---+--------------------+
only showing top 20 rows

14070336 users in sample 2 Dec 2018
+---+--------------------+
|cnt|             percent|
+---+--------------------+
|  1|   94.60975203435085|
|  2|   4.866607307743042|
|  3|  0.4070549559015506|
|  4| 0.07019732862100805|
|  5|0.020958987759780577|
|  6|0.008834188465719654|
|  7| 0.004

## This is older experiment, spliting users by country. This query was done in May, 2018. 
`
* Date considered is 2018/05/01

In [2]:
from pyspark.sql.types import LongType

def hashing(a):
    return hash(a)

results = {}
sqlContext.udf.register("hashingPy", hashing, LongType())
for country in ['FR','DZ']:
    print(country)
    ## QUERY FOR A GIVEN DAY (2018/05/01). 
    query = '''
    SELECT hashingPy(CONCAT(user_agent,client_ip)) as user,   normalized_host.project, COUNT(normalized_host) as projectCount
    FROM webrequest WHERE normalized_host.project_class = "wikipedia" AND is_pageview = 1 AND agent_type="user" 
    AND year=2018 AND month = 5 and day = 10 AND SUBSTR(ip,-1,1) = 5 AND geocoded_data.country_code = "%s" AND access_method = "desktop"
    GROUP BY user, normalized_host
    ''' % country
    df = sqlContext.sql(query)
    sqlContext.registerDataFrameAsTable(df, "users")
    dfUsers = sqlContext.sql('SELECT user,COUNT(project) as cnt FROM users GROUP BY user SORT BY cnt')
    sqlContext.registerDataFrameAsTable(dfUsers, "usersCounts")    
    totalUsers = dfUsers.count()
    print(totalUsers)
    final = sqlContext.sql('SELECT cnt,100*COUNT(cnt)/%s  as percent FROM usersCounts GROUP BY cnt SORT BY cnt DESC' % totalUsers)
    final.sort('cnt').show()
    results[country] = final.toPandas()

FR
185887
+---+--------------------+
|cnt|             percent|
+---+--------------------+
|  1|   89.62003798006316|
|  2|   9.249705466224103|
|  3|  0.8510546730002636|
|  4| 0.15654671924341132|
|  5|0.042498937526561836|
|  6|0.024208255553104845|
|  7|0.013449030862836024|
|  8|0.009683302221241937|
|  9|0.005379612345134...|
| 10|0.001613883703540323|
| 11|0.001613883703540323|
| 12|0.003227767407080646|
| 13|0.001075922469026882|
| 14|0.002151844938053764|
| 16|0.001613883703540323|
| 17| 5.37961234513441E-4|
| 18|0.001075922469026882|
| 19| 5.37961234513441E-4|
| 24| 5.37961234513441E-4|
| 27|0.001613883703540323|
+---+--------------------+
only showing top 20 rows

DZ
10634
+---+--------------------+
|cnt|             percent|
+---+--------------------+
|  1|   87.78446492382923|
|  2|  10.334775249200677|
|  3|  1.6174534511942824|
|  4| 0.21628738010156104|
|  5|0.018807598269700958|
|  6|0.009403799134850479|
| 14|0.009403799134850479|
| 94|0.009403799134850479|
+---+-----

In [6]:
#for all countries
from pyspark.sql.types import LongType

def hashing(a):
    return hash(a)

results = {}
sqlContext.udf.register("hashingPy", hashing, LongType())
for country in ['BE']:
    ## QUERY FOR A GIVEN DAY (2018/05/01). The information for the same days of the survey is not currently available
    ## sampling 10% of users
    query = '''
    SELECT hashingPy(CONCAT(user_agent,client_ip)) as user,   normalized_host.project, COUNT(normalized_host) as projectCount
    FROM webrequest WHERE normalized_host.project_class = "wikipedia" AND is_pageview = 1 AND agent_type="user" 
    AND year=2018 AND month = 12 and day = 14 AND SUBSTR(ip,-1,1) = 5 AND access_method = "desktop"
    GROUP BY user, normalized_host
    ''' 
    df = sqlContext.sql(query)
    sqlContext.registerDataFrameAsTable(df, "users")
    dfUsers = sqlContext.sql('SELECT user,COUNT(project) as cnt FROM users GROUP BY user SORT BY cnt')
    sqlContext.registerDataFrameAsTable(dfUsers, "usersCounts")    
    totalUsers = dfUsers.count()
    print(totalUsers)
    final = sqlContext.sql('SELECT cnt,100*COUNT(cnt)/%s  as percent FROM usersCounts GROUP BY cnt SORT BY cnt DESC' % totalUsers)
    final.sort('cnt').show()
    results[country] = final.toPandas()

3602264
+---+--------------------+
|cnt|             percent|
+---+--------------------+
|  1|   91.09659924980512|
|  2|   7.741492572448882|
|  3|   0.827701689826176|
|  4| 0.17502881521176683|
|  5| 0.06484810663516055|
|  6| 0.02970354199470111|
|  7| 0.01676723305121446|
|  8|0.009799392826289244|
|  9|0.007717368854698...|
| 10|0.006051749677425086|
| 11|0.003830924107727...|
| 12|0.002748271642500...|
| 13|0.001998743012727...|
| 14|0.001276974702575...|
| 15|9.160905475001277E-4|
| 16|7.495286297728317E-4|
| 17|6.662476709091838E-4|
| 18|3.886444746970239E-4|
| 19|6.940079905303997E-4|
| 20|4.719254335606718...|
+---+--------------------+
only showing top 20 rows

